In [ ]:
# comment out to run on drive + colab

# ! pip install transformers_interpret

In [ ]:
# from google.colab import drive
#
# drive.mount('/content/drive/')

In [ ]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import MultiLabelClassificationExplainer
import torch
from transformers import pipeline
from datasets import load_from_disk

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# comment out the filepath to run on drive + colab
filepath = "../models/scenario_level/anomic_conduction"
# filepath = "/content/drive/Shareddrives/AphasiaProject/models/scenario_level/anomic_conduction"

model = AutoModelForSequenceClassification.from_pretrained(filepath+"/model")
tokenizer = AutoTokenizer.from_pretrained(filepath+"/tokenizer", padding=True,
                                          truncation=True, return_tensors="pt",
                                          add_special_tokens=True, max_length=512)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == torch.device("cuda"):
    model.to(device)
pipe = pipeline("text-classification", model=filepath+"/model", tokenizer=filepath+"/tokenizer", device=device, truncation=True, padding=True)
cls_explainer = MultiLabelClassificationExplainer(model, tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model.config

DistilBertConfig {
  "_name_or_path": "../models/anomic_conduction/model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "ANOMIC",
    "1": "CONDUCTION"
  },
  "initializer_range": 0.02,
  "label2id": {
    "ANOMIC": 0,
    "CONDUCTION": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.39.0.dev0",
  "vocab_size": 30524
}

In [ ]:
model.config.id2label

{0: 'ANOMIC', 1: 'CONDUCTION'}

In [ ]:
def interpret(data):
    for text, label in zip(list(data["new_preprocessed_text"]), list(data["label"])):
        if device == torch.device("cuda"):
            inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True, max_length=512).input_ids.to(device)
            # tranformers interpret gives error for input of 512 tokens
            if len(inputs[0]) != 512:
                with torch.no_grad():
                    logits = model(inputs).logits

                predicted_class_id = logits.argmax().item()
                print("True: ", label, "Pred: ", model.config.id2label[predicted_class_id])
                print("Text: ", text)
                cls_explainer(text)
                cls_explainer.visualize()
                print("----------------------------------------------------------------------")
        else:
            inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True, max_length=512)
            # tranformers interpret gives error for input of 512 tokens
            if len(inputs[0]) != 512:
                with torch.no_grad():
                    logits = model(**inputs).logits

                predicted_class_id = logits.argmax().item()
                print("True: ", label, "Pred: ", model.config.id2label[predicted_class_id])
                print("Text: ", text)
                cls_explainer(text)
                cls_explainer.visualize()
                print("----------------------------------------------------------------------")

In [ ]:
scenarios = ["Speech", "Important_Event", "Cinderella", "Stroke", "Cat"]
# make sure not to interpret on trained scenarios

texts = load_from_disk(filepath + "/dataset")["test"].remove_columns(["Unnamed: 0", "input_ids", "attention_mask"]).to_pandas()
texts["label"] = [model.config.id2label[x] for x in texts["label"]]
n = 3

# Speech scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[0])].sample(n=n)])

In [ ]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  [FP] [FP] [FP] [FP] [FP] slowly recovering. [FP] i go to the bar and [FP] and fast eddie's bar and [FP] play [FP] video games of poker. and [FP] once in awhile i i have a beer and [FP] tea. [FP] i'll drink tea. and [FP] [FP] [FP] [FP] firstnames my wife [FP] is a delta employee [FP] no [FP] retired and [FP] [FP] firstnamed lastname [FP] well a lobbyist [FP] a secretary for a lobbyist [FP] in downtown dc.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  no. yeah. yeah. yeah.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.75),ANOMIC,2.59,[CLS] no . yeah . yeah . yeah . [SEP]
n/a,(0.28),CONDUCTION,-2.57,[CLS] no . yeah . yeah . yeah . [SEP]


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  mhm. mhm. it's not very well. i hafta i hadta go to speech and learn how to talk. yeah. uhhuh.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  ANOMIC
Text:  i get i get i just hafta make my one in the. one one a. but that that that's about it.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  ANOMIC
Text:  i want to be in [FP] all da way better. and i want to [FP] really working on my speech. i want to work on. and [FP] my thoughts come [FP] out and and me more and more.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  ANOMIC
Text:  it's fine. ah fine.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.67),ANOMIC,2.26,[CLS] it ' s fine . ah fine . [SEP]
n/a,(0.33),CONDUCTION,-2.32,[CLS] it ' s fine . ah fine . [SEP]


----------------------------------------------------------------------


# Important event scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[1])].sample(n=n)])

In [ ]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  and we were go walking to the around the mall. and we went to the pet score. and there's one dog. it it too much money. it was four hundred dollars. and finally [FP] the dog dog would [FP] get out of the cage and wreck the store. and they didn't know how that he's getting out its cage. so the guy says finally said. you want him? take him. and he got the dog for nothing. yeah. yeah and it was a good dog. i used to take him drinking all the time. no! the the [FP] owner of the place would give it [FP] give him steaks. it was outside. no it was a patio. it was. sasha.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  i [FP] [FP] every sunday or every other sunday. and [FP] [FP] [FP] well [FP] i was pretty good when i was [FP] thirty. but i i [FP] i aim to go back. just when by bike was over my head. oh it was accident. [FP] maybe [FP] lots of times. of corcourse [FP] i'm riding every sunday or every other sunday. and [UP] is [FP] it's pretty good. [FP] bike is fix [FP] forget it. guess yourself up. ride on.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  and i then went to the marine corp and was went through training to become a second lieutenant. and i was a second lieutenant in vietnam. and [FP] luckily i made it. i lived through it. and now that that was pretty much my college and freshman year. i mean i graduated college then going in the marine corp. i became a lieutenant because i i was gonna get draft i was gonna get drafted. so i enlisted. and then i ended up getting out early because they they we kicked we pulled out of vietnam. so i was in vietnam for about ten months as a lieutenant and a captain. oh i the year i graduated college was the peak year of the draft. so i won. but i would've been drafted up to two hundred and twenty. so two thirds of people got drafted that year.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  ANOMIC
Text:  okay. well [FP]. my mouth is [FP] sealed okay. oh yeah is [FP] lot. well is [FP] older and [FP] [FP] [FP] older guy [FP] me is [FP] little kid? okay. alright. i think [FP] me is [FP] five six seven eight nine years old and baptized. i thinking. okay. my sister and me is [FP] baptized at [FP] each at [FP] [FP] [FP] no sunday morning is [FP] guy kaski baptized [FP] me. because i i know [FP] god is ruled and [FP] this and this and this. i know it. is [FP] good. it's alright. it's [FP] good.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  ANOMIC
Text:  and she lost the blood. a lot of it. [FP] anyway i have a lot of respect for my wife. we have three daughters. [FP] i love them. okay? [FP] [FP] well that's it. yeah. yeah.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  CONDUCTION
Text:  he's a [FP] what do you call that a farmer guy up there. [FP] outside desoto. and my dad bought me a a bike. one vroom vroom one of them. i was think i was eleven. i we took by the river kind of, driving back forth. and or speedy had a party. it was [FP] the fourth. and [FP] when we come back the this bike it was stuck. you couldn't we couldn't stop. and we had a wreck. i got hurt a little bit but my dad was in the hospital for [FP] three months and stuff. he hit. look i was on i was on top of him. so i didn't get much. it just cut me here. cuts you know. but anyway always remember that. and i never got another bike. my dad when he come back. it was three days. my dad didn't know who was. he thought my mom [UP] my dad thought it was sister my mom you know. my mom's crying all the time because she thought he'd be like that forever you know. but anyway he come back too. becau

----------------------------------------------------------------------


# Cinderella scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[2])].sample(n=n)])

In [ ]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  i mean [FP] she [FP] she's okay. she's she's just a maid. that's all. finally they the sisters and the mother get to go to see a dance that the prince gives. and off she goes to the dance. and the the mice tell her [FP]. midnight you must be home. okay. so she she goes, has a wonderful time, but it's midnight and she goes off to the [FP] the coach and goes home. i mean her. dress and all. well anyhow only thing she she has [FP] lost a slipper. so that's all. then he hasta know he is the [UP] prince. he must find who [FP] law slipper. and they finally come to her house and her her [FP] the daughter's there. they [FP] they don't fit the slippers, but cinderella does. that's it.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  she met a a nice person. and she married him. and then at the balance she always she goes away with a nice man.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  and he took her to live with [FP] godmother. and he then died. and the godmother treated cinderella badly and made her work for the two daughters and herself. [FP] she hadta clean up all the mess and make [FP] make do with the farm animals. the mice put put her together for her. and [FP] her sisters saw it and tore it up. and she went out into the garden. and she played [FP] she cried to to them. and her fairy godmother made [FP] a [FP] the [FP] wagon to [FP] [FP] [FP] [FP] made her get to the ball. and she was on time and in in a good humor. and all she danced with the prince [FP] up to the hour of twelve o'clock which was when [FP] [FP] her fairy godmother told her to be back. and she she ran out of the [FP] castle with [FP] dropping her foot or one glove her one [FP] shoe. and she got the got home. and she knew that she could pair the the missing shoe to the one she had. and she

----------------------------------------------------------------------
True:  CONDUCTION Pred:  CONDUCTION
Text:  the the other she she didn't really like her girl. the the she had two girls to her marriage first. so she took then [FP] the cinderella was [FP] always he was always leave over his he was go he no she was going going to the ball. but they didn't let they didn't wanna want her to. then anyway the [FP] guy that that she go to the ball. and he took to to to the [FP] the ball. and [FP] the the prince really liked him. and then they were til twelve. and he had left. he said he he said. don't. he said. i don't wanna go. but she wanna she home. and the the the the press prince she went to the every house and found her his his cinderella. no. [UP] she went to he married in the end.


----------------------------------------------------------------------


# Stroke scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    df = texts[(texts["label"] == x) & (texts["scenario"] == scenarios[3])]
    # no controls in stroke scenario
    if len(df) >= n:
        data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[3])].sample(n=n)])

In [ ]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  [FP] [FP] i i in the hospital. yeah. three or four days. yes. [FP] [FP] what's name? scale. scale. [FP] what name? i don't know.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  yes. [UP] november thirty oh twenty two thousand and five i had a stroke at work. [UP] i didn't know that i had a stroke. [UP] a colleague [UP] drop off a letter for me to review. and he saw that i was on the floor. [UP] it was about four hours after the stroke. yes. i [UP] i [UP] was evacuated to the hospital in. and then i [UP] the nets hospital. [UP] in hospital. [UP] i was helicopter there. and [UP] my [UP] my no wait my wipe. [UP] one there for the for me. yeah. i was in there for [UP] four five six seven seven days. and then i went to the mount vernon hospital for rehab work. i was in there for four four weeks and a [UP] little bit more. [UP] boy. [UP] boy. what kind of things that you were interested in? [UP] oh boy. [UP] mount vernon rehab i [UP] i add a speech. i add a [UP] arm and i had a leg. [UP] this was completed in about three or four months. in [UP] i had a repeat o

----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  like this. no. look. yes. [FP] about five years ago. i don't know. i didn't feel [FP] right. i called my friend, and he [FP] she, [FP] called the fire department. and that was it. i went to the hospital. i went to the the health south. they were wonderful. [FP] [UP] i didn't have this [FP] business with my hips then, or whatever it is. and i [FP], [FP] therapy for my leg and arm. [FP] and did [FP] they also [FP] taught me to a a therapist for my speech. and she was terrible! and she didn't ask me thing what i what i wanted [FP] [FP] what i wanted to do. and [FP] i say didn't i didn't didn't [FP] see her when i did but i didn't. she was terrible. and then from there i went to cooley dick to a speech therapist who was wonderful. and she retired. right. things i hadta do at home and so forth. he that she said. no [FP] we're gonna do this that and that. and they weren't sort of [FP] ki

----------------------------------------------------------------------
True:  CONDUCTION Pred:  CONDUCTION
Text:  nothing. [FP] no, no, yes. the center at the stroke no good. the [FP] the right down there is another one. no good. i don't no talking no good talking. and the i don't know where is begin. that one was a good one. yes. there okay okay. the center was a good one. was a good one. a camera the camera. oh my yes and i like the big one. [FP] [FP] the camera. i all kinds of all kinds. no no no no. oh long. yes a long time afterwards.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  ANOMIC
Text:  well [FP] medchi useta work. [FP] this evening. and headache. and [FP] sit down. aspirin. no. and talking. and wobbly. and a stroke. yeah. the coma. yeah. [FP] red. [FP] [UP] one month. and [FP] patti labelle [FP] somewhere over the rainbow. i woke up. yeah. [FP] we write sentence sentences. and [FP] arts. and [FP] [UP] the wii. [FP] singing helps. [FP] [FP] pictures. [FP] [UP] [FP] [UP] [FP] we baseball. we took a bus. and the orioles and football. yeah.


----------------------------------------------------------------------


# Cat scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [ ]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[4])].sample(n=n)])

In [ ]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  this one's a tough one. a a a. not a dog. cat. trying to race cat down [FP] down the down [FP] wants him come down. with the and he he wants to wants to rode his rode his bike bicycle. he share a bicycle over to get get his cat. [FP] the young man is up the tree being chased by a by a dog. [FP] keeping him keep him up the thing. and a a bird watching singing. [FP] with the [FP] two gentleman with a [FP] ice going over to the thing to either either bring bring the gentleman up with the dog or the girl with the [FP] with the [FP] dog no cat. and and watching coming over. and ladder labor day. that's one of them i can't get. i can't figure it out. i know i know what is it. i got one at home. go up and down! ladder! it's a ladder. finally. and a ladder. and they brought the ladder over to bring up either the dog or get the dog away to keep the man or save the save the kitten with the with the with the daughter.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  [UP] okay. so [FP] the the man [FP] [UP] started [FP] [UP] [FP] the [UP] so the [FP] so the [UP] so the girl and the man [FP] [UP] [FP] they saw that the kitty cat was [FP] in the tree. [FP] and so the man went and got a ladder. [FP] and he went [FP] up to the tree and tried to get the kitty cat. and [FP] but the the the ladder actually fell. [FP] and so he [UP] he [FP] got the firemen rescues. and [FP] [UP] tried to [FP] [UP] so he's just being rescued.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  okay. oh okay. [FP] okay. the daughter was happy because the father is [FP] rescued the cat. [FP] the also the firemen are going to pick him up. the the father is heights there the father is a is a [FP] heights. the father is afraid of heights heights. okay. so the guy the fireman has two dmz two jobs, the cat and the [FP] father.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  ANOMIC
Text:  okay. well this this girl is standing under the tree. and she's about that's in the tree. he's there on the branch. and he went to sleep again right there. okay. and [FP] lie to get him up there. he didn't he didn't get up there. so he he stayed here the boy. but he got you gotta get the cat out of there. and then with that [FP] they got the [FP] firetruck over. and then they got that. and they got the tree.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  ANOMIC
Text:  i'm looking for the [FP] for the [FP] [FP] the the thing that's sposta be. anyways she was hanging over. saw saw it fell fell down. she stuck with it. the dog is looking up. [FP] he'd been barking. the they probably what it was. they came over and realized what they. first they came came they came to the there to get the to get the cat. and then a hook it he realized see saw that he fell off he fell over that. he was stuck without it. so the the [FP] the the this little girl's up there top [FP] [FP] stopped up there. [FP] she wanted to be [FP] she wants the cat. waiting to come come back. she wanted to come back for the father. [FP] she's she's over there [FP] you know with her [FP] with her bike. and [FP] sitting there. and i'm i'm i'm [FP] actually i'm i'm seeing seeing more than what they're saying.


----------------------------------------------------------------------
True:  CONDUCTION Pred:  ANOMIC
Text:  i've seen this before! i've seen this picture! you know me. i it's bad on me. so it's a girl. and this girl is [FP] a bicycle girl. she has a bicycle. and she's had her bicycle for herself. and she sees her cat. i i'm saying the right words? you got a cat. and the cat is up in the [FP] up in the wall. i'm saying the wrong word. but on this thing here the wall or whatever you call it. and and what's happening is she has a dog. and the dog is up on the tree. do i said the right word? the fence or tree. i guess a tree you call it. and her father a guy a guy is standing here. and he's got his hands on the [FP] deal. somehow he got up here. and he and he's got on the deal. and he's trying to get a chair the desk table. table i table l e b [FP] label table. and she wants to bring that table to bring it up here to put on here. and [FP] she got a couple of animal birds [FP] birds here.

----------------------------------------------------------------------
